# Trabajo Práctico Nº3 - Big Data

### Mateo Servent, Joaquín Musich y Andres Cuellas

### Parte I: Análisis de la base de hogares y cálculo de pobreza

In [1]:
import pandas as pd

# Leer la base de datos
df = pd.read_excel('/Users/macbook/Desktop/UdeSa/Materias/Big data/Big-Data/TP3/usu_hogar_T123.xlsx')

# Estadísticos descriptivos (más la exploración del libro de codigos)
 
df.describe().T

FileNotFoundError: [Errno 2] No such file or directory: '/Users/macbook/Desktop/UdeSa/Materias/Big data/Big-Data/TP3/usu_hogar_T123.xlsx'

### A partir de un análisis preliminar, consideramos que posibles determinantes claves en la predicción de la pobreza dentro del conjunto de datos de hogares serían los atributos vinculados a la infraestructura habitacional (calidad estructural, conectividad a servicios básicos, instalaciones sanitarias), el esquema de propiedad o alquiler del inmueble y la distribución demográfica por edad en el núcleo familiar. Estos indicadores, podrían llegar a ser considerados esenciales para una estimación más afinada.

In [24]:
# 2)
# Filtrar observaciones
df_filtrado = df[df['AGLOMERADO'].isin([32, 33])]

,CODUSU,NRO_HOGAR,COMPONENTE,H15,CH03,CH04,CH05,CH06,CH07,CH08,...,GDECCFR,PDECCFR,ADECCFR,PONDIH,VII1_1,VII1_2,VII2_1,VII2_2,VII2_3,VII2_4
0,TQRMNORUYHLMSMCDEIJAH00719364,1,1,1,1,2,1957-10-17 00:00:00,65,1,1.0,...,5.0,NaN,6,2540,1,0,2,0,0,0
1,TQRMNORUYHLMSMCDEIJAH00719364,1,2,1,2,1,1952-04-07 00:00:00,70,1,1.0,...,5.0,NaN,6,2540,1,0,2,0,0,0
2,TQRMNOSRQHJNSOCDEIJAH00802640,1,1,1,1,1,1986-09-26 00:00:00,36,1,4.0,...,1.0,NaN,1,14082,2,1,98,0,0,0
3,TQRMNOSRQHJNSOCDEIJAH00802640,1,2,1,2,2,1994-05-02 00:00:00,28,1,4.0,...,1.0,NaN,1,14082,2,1,98,0,0,0
4,TQRMNOSRQHJNSOCDEIJAH00802640,1,3,0,3,2,2014-03-20 00:00:00,9,5,4.0,...,1.0,NaN,1,14082,2,1,98,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7614,TQRMNOPVVHKLRQCDEIJAH00780546,1,3,0,3,1,2013-08-05 00:00:00,9,5,2.0,...,9.0,NaN,9,2295,1,2,98,0,0,0
7615,TQRMNOQPTHKKRRCDEIJAH00780327,1,1,1,1,2,1945-06-05 00:00:00,77,3,1.0,...,10.0,NaN,10,1516,1,0,96,0,0,0
7616,TQRMNOPTXHKORSCDEIJAH00780269,1,1,1,1,2,1977-07-31 00:00:00,45,5,1.0,...,12.0,NaN,12,0,1,2,3,0,0,0
7617,TQRMNOPTXHKORSCDEIJAH00780269,1,2,1,3,1,2005-06-17 00:00:00,17,5,1.0,...,12.0,NaN,12,0,1,2,3,0,0,0


In [10]:
# 3)

dfI = pd.read_excel('/Users/macbook/Desktop/UdeSa/Materias/Big data/Big-Data/TP2/EPH_usu_1er_Trim_2023/usu_individual_T123.xlsx')

dfI = dfI[dfI['AGLOMERADO'].isin([32, 33])]

# Definimos las intersección entre las dos bases como una lista
columnas_duplicadas = set(dfI.columns).intersection(set(df_filtrado.columns))
# Removemos CODUSU y NRO_HOGAR de la lista ya que queremos usarlas para la intersección 
#(y las columnas sobre las que se une no generan duplicados)
columnas_duplicadas.remove("CODUSU")
columnas_duplicadas.remove("NRO_HOGAR")

# Hacemos el merge habiendo dropeado las columnas repetidas
df2 = pd.merge(
    dfI.drop(columnas_duplicadas, axis=1), 
    df_filtrado,
    on= ['NRO_HOGAR', 'CODUSU'])

In [25]:
# 4 y 5 ) 
def eliminar_categoria_9(df, column_list):
    """
    Elimina la categoría con valor 9 de una serie de variables en un DataFrame.
    
    Parámetros:
    - df: DataFrame donde se encuentran las columnas.
    - column_list: Lista de columnas en las que se quiere eliminar la categoría con valor 9.
    
    Retorna:
    - DataFrame con las categorías con valor 9 eliminadas en las columnas especificadas.
    """
    for column in column_list:
        df = df[df[column] != 9]
    return df


df3 = eliminar_categoria_9(df2, ['NIVEL_ED', 'CH08', 'CH07', 'CAT_OCUP', 'CH15', 'CH16']) # Hay más NAs en otras variables
df4 = df3[(df3['CH06'] > 0)] # Edad menor a cero

In [22]:
def columnas_duplicadas(df):
    """
    Esta función verifica si hay columnas con valores idénticos en un DataFrame.
    
    Argumentos:
    - df: DataFrame de pandas
    
    Retorna:
    - Una lista de tuplas con los nombres de las columnas repetidas.
    """
    
    columnas_repetidas = []
    
    for i in range(df.shape[1]):
        for j in range(i+1, df.shape[1]):
            if df.iloc[:, i].equals(df.iloc[:, j]):
                columnas_repetidas.append((df.columns[i], df.columns[j]))
    
    return columnas_repetidas


def eliminar_columnas_duplicadas(df):
    # Obtener las columnas duplicadas
    cols_duplicadas = columnas_duplicadas(df)
    
    # Crear una lista de columnas para eliminar
    columnas_para_eliminar = [col[1] for col in cols_duplicadas]
    
    # Eliminar las columnas
    df = df.drop(columns=columnas_para_eliminar)
    
    return df

In [28]:
eliminar_columnas_duplicadas(df4) # verificamos si hubiera columnas duplicadas y las eliminamos

def drop_high_na(df, threshold=0.7):
    """
    Elimina las columnas y las filas del DataFrame que tienen un porcentaje de valores faltantes 
    superior al umbral especificado.

    Parámetros:
    - df: DataFrame de entrada.
    - threshold: umbral de porcentaje de valores faltantes para eliminar una columna o fila (valor predeterminado = 0.7).

    Retorna:
    - DataFrame después de eliminar las columnas y filas con un alto porcentaje de valores faltantes.
    """
    # Calcula el porcentaje de valores faltantes para cada columna
    missing_percentage_col = df.isnull().mean()
    
    # Filtra las columnas que tienen un porcentaje de valores faltantes superior al umbral
    columns_to_drop = missing_percentage_col[missing_percentage_col > threshold].index
    
    # Elimina las columnas seleccionadas del DataFrame
    df_cleaned = df.drop(columns=columns_to_drop)
    
    # Calcula el porcentaje de valores faltantes para cada fila
    missing_percentage_row = df_cleaned.isnull().mean(axis=1)
    
    # Filtra las filas que tienen un porcentaje de valores faltantes superior al umbral
    rows_to_drop = missing_percentage_row[missing_percentage_row > threshold].index
    
    # Elimina las filas seleccionadas del DataFrame
    df_cleaned = df_cleaned.drop(index=rows_to_drop)
    
    return df_cleaned

drop_high_na(df4)

,CODUSU,NRO_HOGAR,COMPONENTE,H15,CH03,CH04,CH05,CH06,CH07,CH08,...,RDECCFR,GDECCFR,ADECCFR,PONDIH,VII1_1,VII1_2,VII2_1,VII2_2,VII2_3,VII2_4
0,TQRMNORUYHLMSMCDEIJAH00719364,1,1,1,1,2,1957-10-17 00:00:00,65,1,1.0,...,5,5.0,6,2540,1,0,2,0,0,0
1,TQRMNORUYHLMSMCDEIJAH00719364,1,2,1,2,1,1952-04-07 00:00:00,70,1,1.0,...,5,5.0,6,2540,1,0,2,0,0,0
2,TQRMNOSRQHJNSOCDEIJAH00802640,1,1,1,1,1,1986-09-26 00:00:00,36,1,4.0,...,1,1.0,1,14082,2,1,98,0,0,0
3,TQRMNOSRQHJNSOCDEIJAH00802640,1,2,1,2,2,1994-05-02 00:00:00,28,1,4.0,...,1,1.0,1,14082,2,1,98,0,0,0
4,TQRMNOSRQHJNSOCDEIJAH00802640,1,3,0,3,2,2014-03-20 00:00:00,9,5,4.0,...,1,1.0,1,14082,2,1,98,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7614,TQRMNOPVVHKLRQCDEIJAH00780546,1,3,0,3,1,2013-08-05 00:00:00,9,5,2.0,...,9,9.0,9,2295,1,2,98,0,0,0
7615,TQRMNOQPTHKKRRCDEIJAH00780327,1,1,1,1,2,1945-06-05 00:00:00,77,3,1.0,...,10,10.0,10,1516,1,0,96,0,0,0
7616,TQRMNOPTXHKORSCDEIJAH00780269,1,1,1,1,2,1977-07-31 00:00:00,45,5,1.0,...,12,12.0,12,0,1,2,3,0,0,0
7617,TQRMNOPTXHKORSCDEIJAH00780269,1,2,1,3,1,2005-06-17 00:00:00,17,5,1.0,...,12,12.0,12,0,1,2,3,0,0,0


In [33]:
# 6) Estadísticas descriptivas
# IV6: Suministro de agua
# IV3: Tipos de piso interior
# IV4: Tipo de cubierta exterior del techo
# IV11: Desague del baño
# IV12_3: vivienda ubicada en una villa de emergencia

df4[['IV3', "IV12_3", 'V4', 'IV6', 'IV11']].describe().T

,count,mean,std,min,25%,50%,75%,max
IV3,7536.0,1.123275,0.347611,1.0,1.0,1.0,1.0,4.0
IV12_3,7536.0,1.994294,0.075327,1.0,2.0,2.0,2.0,2.0
V4,7536.0,2.004910,0.248904,1.0,2.0,2.0,2.0,9.0
IV6,7536.0,1.026407,0.169994,1.0,1.0,1.0,1.0,3.0
IV11,7536.0,1.372346,0.642366,0.0,1.0,1.0,2.0,4.0


#Parte 2


In [1]:
#1)
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import os
import statistics

In [2]:


def evalua_metodo(X_train, X_test, y_train, y_test, model, k=5, landa=1):
    if model == KNeighborsClassifier:
        y_model = model(k).fit(X_train, y_train)
    elif model == LogisticRegression:
        y_model = model(C=1/landa).fit(X_train, y_train)
    elif model == LinearDiscriminantAnalysis:
        y_model = model().fit(X_train, y_train)
    else:
        y_model = model.fit(X_train, y_train)
    
    y_pred = y_model.predict(X_test)

    matriz_confusion = confusion_matrix(y_test, y_pred)
    auc = roc_auc_score(y_test, y_pred)
    fpr, tpr, tresholds = roc_curve(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)

    ecm = mean_squared_error(y_test, y_pred)

    df_metrics = pd.DataFrame({'modelo': [model], 'accuracy': [accuracy], 'auc':[auc], 'ecm':[ecm]})

    return df_metrics


In [5]:
#2) Cross validation model

def cross_validation(model, k, x, y, standard=False):
    sc = StandardScaler()
    kf = KFold(n_splits=k, shuffle=True)
    ecm = []

    for train_index, test_index in kf.split(x):
        x_train, x_test = x.iloc[train_index], x.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        if standard:
            x_train = sc.fit_transform(x_train)
            x_test = sc.transform(x_test)

        ecm2 = evalua_metodo(x_train, x_test, y_train, y_test, model)
        ecm3 = ecm2['ecm'][0]
        ecm.append(ecm3)

    final_ecm = statistics.mean(ecm)

    return final_ecm





In [6]:
#3)

def evalua_config(landa, k, x, y, l1_ratio=1):
    ret = {}
    for elem in landa:
        clf_en_LR = LogisticRegression(
            C=1/elem, penalty="elasticnet", solver="saga", l1_ratio=l1_ratio, tol=0.01)
        ecm = cross_validation(clf_en_LR, k, x, y, standard=True)
        ret[elem] = ecm

    return min(ret, key=ret.get)


In [7]:
#4)
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd
import numpy as np

def evalua_multiples_metodos(k_cv, k_knn, landa_max, x_train, x_test, y_train, y_test, x, y):
    modelos = ['regresion_logistica', 'k_vecinos_cercanos', 'analisis_discriminante']
    landas = np.arange(0.01, landa_max, 10).tolist()

    matriz = pd.DataFrame(columns=["Modelo", "Hiperparametro", "Precisión", "AUC", "ECM"])

    for modelo in modelos:
        if modelo == 'regresion_logistica':
            landa_optimo = evalua_config(landas, k_cv, x, y)  # Hiperparámetro landa
            metricas_log = evalua_metodo(x_train, x_test, y_train, y_test, LogisticRegression, landa=landa_optimo)
            results = [modelo, landa_optimo, metricas_log['accuracy'][0], metricas_log['auc'][0], metricas_log['ecm'][0]]
            matriz.loc[len(matriz)] = results

        elif modelo == 'k_vecinos_cercanos':
            metricas_kvc = evalua_metodo(x_train, x_test, y_train, y_test, KNeighborsClassifier, k=k_knn)
            results = [modelo, k_knn, metricas_kvc['accuracy'][0], metricas_kvc['auc'][0], metricas_kvc['ecm'][0]]
            matriz.loc[len(matriz)] = results

        elif modelo == 'analisis_discriminante':
            metricas_ad = evalua_metodo(x_train, x_test, y_train, y_test, LinearDiscriminantAnalysis)
            results = [modelo, "NA", metricas_ad['accuracy'][0], metricas_ad['auc'][0], metricas_ad['ecm'][0]]
            matriz.loc[len(matriz)] = results

    return matriz
